In [52]:
import sys
sys.path.append("../")
import copy
import re
import pandas as pd
import numpy as np
from experiments import ExperimentModule
from experiments import (
    save_plots, 
    write_row, 
    write_opcounts,
    save_circuit,
    load_qasm_circuit,
    create_dir,
    get_random_state
)
import matplotlib.pyplot as plt
from itertools import product

%matplotlib inline

In [53]:
results_dir = "../results/COBYLA/no_ent/random_states"
run_range = list(range(1, 11))
#qubit_range = [3, 4, 5, 6, 7, 8]
qubit_range = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#qubit_range = [3, 4, 5, 6, 7, 8, 9, 10]

# EPS values are mapped separately
#eps_values = [0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
eps_values = [0.5, 0.1, 0.05, 0.01]

### Random State type

In [54]:
state_type = "random_state"

In [55]:
#[0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
original_results_dict = {
    "num_qubits": [],
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}

modified_results_dict = copy.deepcopy(original_results_dict)

for (r_dir, st_type, num_qubits, eps) in product([results_dir], [state_type], qubit_range, eps_values):
    if not num_qubits in original_results_dict["num_qubits"]: 
        original_results_dict["num_qubits"] += [num_qubits]
        modified_results_dict["num_qubits"] += [num_qubits]

    original_results_dict[eps] += [[]]
    modified_results_dict[eps] += [[]]

    for run_idx in run_range:
        fid_loss_original = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/csv/original_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_original = fid_loss_original[-1][-1] if len(fid_loss_original.shape) > 1 else fid_loss_original[-1]
        original_results_dict[eps][-1] += [data_original.round(4)]

        fid_loss_modified = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/csv/modified_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_modified = fid_loss_modified[-1][-1] if len(fid_loss_modified.shape) > 1 else fid_loss_modified[-1]
        modified_results_dict[eps][-1]+= [data_modified.round(4)]

In [56]:
default_dict = {
    "num_qubits": qubit_range,
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}
mean_original_results_dict = copy.deepcopy(default_dict)
mean_modified_results_dict = copy.deepcopy(default_dict)

stds_original_results_dict = copy.deepcopy(default_dict)
stds_modified_results_dict = copy.deepcopy(default_dict)

for eps in eps_values:
    mean_original_results_dict[eps] = np.mean(original_results_dict[eps], axis=1)
    stds_original_results_dict[eps] = np.std(original_results_dict[eps], axis=1).round(2)

    mean_modified_results_dict[eps] = np.mean(modified_results_dict[eps], axis=1)
    stds_modified_results_dict[eps] = np.std(modified_results_dict[eps], axis=1).round(2)

Original fidloss

In [57]:
df_mean_original = pd.DataFrame(mean_original_results_dict)
df_mean_original = df_mean_original.set_index("num_qubits")
display(df_mean_original)

df_stds_original = pd.DataFrame(stds_original_results_dict)
df_stds_original = df_stds_original.set_index("num_qubits")
df_stds_original

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.05950,0.05950,0.00025,0.00393
4,0.17780,0.17780,0.12397,0.00253
5,0.19080,0.19080,0.17740,0.15288
6,0.22480,0.22480,0.20020,0.17440
7,0.24860,0.24860,0.24340,0.22460
8,0.23700,0.23640,0.71925,0.22630
9,0.25290,0.25290,0.78492,0.69626
10,0.70510,0.72077,0.56216,0.90107
11,0.38219,0.34028,0.67071,0.66036


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.00,0.01
4,0.00,0.00,0.00,0.01
5,0.00,0.00,0.00,0.02
6,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.02,0.00
9,0.00,0.00,0.04,0.05
10,0.03,0.05,0.07,0.02
11,0.07,0.06,0.05,0.05


In [58]:
print(df_mean_original.to_latex(float_format="%.4f"))

\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.0595 & 0.0595 & 0.0003 & 0.0039 \\
4 & 0.1778 & 0.1778 & 0.1240 & 0.0025 \\
5 & 0.1908 & 0.1908 & 0.1774 & 0.1529 \\
6 & 0.2248 & 0.2248 & 0.2002 & 0.1744 \\
7 & 0.2486 & 0.2486 & 0.2434 & 0.2246 \\
8 & 0.2370 & 0.2364 & 0.7193 & 0.2263 \\
9 & 0.2529 & 0.2529 & 0.7849 & 0.6963 \\
10 & 0.7051 & 0.7208 & 0.5622 & 0.9011 \\
11 & 0.3822 & 0.3403 & 0.6707 & 0.6604 \\
12 & 0.9336 & 0.9334 & 0.6288 & 0.8914 \\
13 & 0.7718 & 0.7446 & 0.6757 & 0.9431 \\
14 & 0.9870 & 0.9834 & 0.9285 & 0.9721 \\
15 & 0.9104 & 0.8906 & 0.9603 & 0.9922 \\
\bottomrule
\end{tabular}



Modified fidloss

In [59]:
df_mean_modified = pd.DataFrame(mean_modified_results_dict)
df_mean_modified = df_mean_modified.set_index("num_qubits")
display(df_mean_modified)

df_stds_modified = pd.DataFrame(stds_modified_results_dict)
df_stds_modified = df_stds_modified.set_index("num_qubits")
df_stds_modified

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.0595,0.0595,0.0000,0.00000
4,0.1742,0.1742,0.1237,0.00000
5,0.1827,0.1827,0.7142,0.13850
6,0.2114,0.2114,0.1852,0.71230
7,0.2420,0.2420,0.6072,0.21420
8,0.2331,0.2331,0.2261,0.21200
9,0.2508,0.2508,0.2462,0.24360
10,0.2400,0.2400,0.2386,0.80840
11,0.2521,0.2521,0.8065,0.24860


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0


In [60]:
print(df_mean_modified.to_latex(float_format="%.4f"))

\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.0595 & 0.0595 & 0.0000 & 0.0000 \\
4 & 0.1742 & 0.1742 & 0.1237 & 0.0000 \\
5 & 0.1827 & 0.1827 & 0.7142 & 0.1385 \\
6 & 0.2114 & 0.2114 & 0.1852 & 0.7123 \\
7 & 0.2420 & 0.2420 & 0.6072 & 0.2142 \\
8 & 0.2331 & 0.2331 & 0.2261 & 0.2120 \\
9 & 0.2508 & 0.2508 & 0.2462 & 0.2436 \\
10 & 0.2400 & 0.2400 & 0.2386 & 0.8084 \\
11 & 0.2521 & 0.2521 & 0.8065 & 0.2486 \\
12 & 0.2499 & 0.2499 & 0.2490 & 0.2481 \\
13 & 0.2478 & 0.2478 & 0.6254 & 0.2472 \\
14 & 0.2487 & 0.2487 & 0.2486 & 0.8113 \\
15 & 0.2496 & 0.2496 & 0.2496 & 0.2499 \\
\bottomrule
\end{tabular}



In [61]:
df_mean_original < df_mean_modified


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,False,False,False,False
4,False,False,False,False
5,False,False,True,False
6,False,False,False,True
7,False,False,True,False
8,False,False,False,False
9,False,False,False,False
10,False,False,False,False
11,False,False,True,False


### Random Sparse States

In [62]:
state_type = "random_sparse_state"
density = 0.3

In [63]:
#[0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
original_results_dict = {
    "num_qubits": [],
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}

modified_results_dict = copy.deepcopy(original_results_dict)

for (r_dir, st_type, num_qubits, eps) in product([results_dir], [state_type], qubit_range, eps_values):
    if not num_qubits in original_results_dict["num_qubits"]: 
        original_results_dict["num_qubits"] += [num_qubits]
        modified_results_dict["num_qubits"] += [num_qubits]

    original_results_dict[eps] += [[]]
    modified_results_dict[eps] += [[]]

    for run_idx in run_range:
        fid_loss_original = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/density_{density}/csv/original_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_original = fid_loss_original[-1][-1] if len(fid_loss_original.shape) > 1 else fid_loss_original[-1]
        original_results_dict[eps][-1] += [data_original.round(4)]

        fid_loss_modified = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/density_{density}/csv/modified_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_modified = fid_loss_modified[-1][-1] if len(fid_loss_modified.shape) > 1 else fid_loss_modified[-1]
        modified_results_dict[eps][-1]+= [data_modified.round(4)]

In [64]:
default_dict = {
    "num_qubits": qubit_range,
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}
mean_original_results_dict = copy.deepcopy(default_dict)
mean_modified_results_dict = copy.deepcopy(default_dict)

stds_original_results_dict = copy.deepcopy(default_dict)
stds_modified_results_dict = copy.deepcopy(default_dict)

for eps in eps_values:
    mean_original_results_dict[eps] = np.mean(original_results_dict[eps], axis=1)
    stds_original_results_dict[eps] = np.std(original_results_dict[eps], axis=1).round(2)

    mean_modified_results_dict[eps] = np.mean(modified_results_dict[eps], axis=1)
    stds_modified_results_dict[eps] = np.std(modified_results_dict[eps], axis=1).round(2)

In [65]:
df_mean_original = pd.DataFrame(mean_original_results_dict)
df_mean_original = df_mean_original.set_index("num_qubits")
display(df_mean_original)
display(df_mean_modified)

df_stds_original = pd.DataFrame(stds_original_results_dict)
df_stds_original = df_stds_original.set_index("num_qubits")
#df_stds_original

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.81787,0.92879,0.01870,0.00000
4,0.85176,0.89916,0.96231,0.00000
5,0.51644,0.51655,0.42050,0.32266
6,0.66893,0.65643,0.60809,0.52517
7,0.65540,0.65540,0.77570,0.59976
8,0.77128,0.75424,0.75245,0.71184
9,0.75630,0.75630,0.93498,0.82497
10,0.77176,0.77177,0.77694,0.95093
11,0.77547,0.77528,0.78511,0.84906


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.0595,0.0595,0.0000,0.00000
4,0.1742,0.1742,0.1237,0.00000
5,0.1827,0.1827,0.7142,0.13850
6,0.2114,0.2114,0.1852,0.71230
7,0.2420,0.2420,0.6072,0.21420
8,0.2331,0.2331,0.2261,0.21200
9,0.2508,0.2508,0.2462,0.24360
10,0.2400,0.2400,0.2386,0.80840
11,0.2521,0.2521,0.8065,0.24860


In [66]:
display(df_stds_original)
display(df_stds_modified)

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.28,0.08,0.04,0.00
4,0.07,0.05,0.05,0.00
5,0.00,0.00,0.00,0.02
6,0.04,0.00,0.01,0.01
7,0.00,0.00,0.06,0.00
8,0.07,0.05,0.05,0.02
9,0.00,0.00,0.01,0.03
10,0.00,0.00,0.01,0.01
11,0.00,0.00,0.01,0.02


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0


In [67]:
print("--"*100)
print("Original Mean")
print("--"*100)
print(df_mean_original.to_latex(float_format="%.4f"))

print("--"*100)
print("Modified Mean")
print("--"*100)
print(df_mean_modified.to_latex(float_format="%.4f"))

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Original Mean
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.8179 & 0.9288 & 0.0187 & 0.0000 \\
4 & 0.8518 & 0.8992 & 0.9623 & 0.0000 \\
5 & 0.5164 & 0.5165 & 0.4205 & 0.3227 \\
6 & 0.6689 & 0.6564 & 0.6081 & 0.5252 \\
7 & 0.6554 & 0.6554 & 0.7757 & 0.5998 \\
8 & 0.7713 & 0.7542 & 0.7524 & 0.7118 \\
9 & 0.7563 & 0.7563 & 0.9350 & 0.8250 \\
10 & 0.7718 & 0.7718 & 0.7769 & 0.9509 \\
11 & 0.7755 & 0.7753 & 0.7851 & 0.8491 \\
12 & 0.9455 & 0.9473 & 0.8810 & 0.9790 \\
13 & 0.9153 & 0.9020 & 0.9124 & 0.9668 \\
14 & 0.9969 & 

In [68]:
df_mean_original < df_mean_modified

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,False,False,False,False
4,False,False,False,False
5,False,False,True,False
6,False,False,False,True
7,False,False,False,False
8,False,False,False,False
9,False,False,False,False
10,False,False,False,False
11,False,False,True,False
